In [ ]:
import numpy as np # linear algebra
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tensorflow.keras.applications import densenet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.densenet import preprocess_input, decode_predictions
import numpy as np
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import os

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_datagen.flow_from_directory('/content/drive/MyDrive/train', 
 target_size=(224,224), 
 color_mode='rgb', 
 batch_size=32, 
 class_mode='categorical', 
 shuffle=True)
category_dict = train_generator.class_indices
print(category_dict)
import tensorflow as tf 
print(tf.__version__)

Found 3581 images belonging to 36 classes.
{'apple': 0, 'banana': 1, 'beetroot': 2, 'bell pepper': 3, 'cabbage': 4, 'capsicum': 5, 'carrot': 6, 'cauliflower': 7, 'chilli pepper': 8, 'corn': 9, 'cucumber': 10, 'eggplant': 11, 'garlic': 12, 'ginger': 13, 'grapes': 14, 'jalepeno': 15, 'kiwi': 16, 'lemon': 17, 'lettuce': 18, 'mango': 19, 'onion': 20, 'orange': 21, 'paprika': 22, 'pear': 23, 'peas': 24, 'pineapple': 25, 'pomegranate': 26, 'potato': 27, 'raddish': 28, 'soy beans': 29, 'spinach': 30, 'sweetcorn': 31, 'sweetpotato': 32, 'tomato': 33, 'turnip': 34, 'watermelon': 35}
2.4.1


In [ ]:
number_of_classes = len(category_dict)
base_model =densenet.DenseNet121(weights='imagenet',include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
preds = Dense(number_of_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=preds)
n_freeze = 300
for layer in model.layers[:n_freeze]:
 layer.trainable=False
for layer in model.layers[n_freeze:]:
 layer.trainable=True

29089792/29084464 [==============================] - 0s 0us/step


In [ ]:
print(len(model.layers))
model.summary()

432
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, None, None, 6 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, None, None, 6 256         conv1/conv[0][0]                 
__________________________________________________________________________________________

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
step_size_train = train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator, 
 steps_per_epoch=step_size_train, 
 epochs=10)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
  1/111 [..............................] - ETA: 44:15 - loss: 3.6672 - accuracy: 0.0312

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


 21/111 [====>.........................] - ETA: 15:01 - loss: 3.1305 - accuracy: 0.2133

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


111/111 [==============================] - 1154s 10s/step - loss: 2.0988 - accuracy: 0.4549
Epoch 2/10
111/111 [==============================] - 131s 1s/step - loss: 0.7528 - accuracy: 0.7689
Epoch 3/10
111/111 [==============================] - 131s 1s/step - loss: 0.5626 - accuracy: 0.8037
Epoch 4/10
111/111 [==============================] - 132s 1s/step - loss: 0.3802 - accuracy: 0.8691
Epoch 5/10
111/111 [==============================] - 132s 1s/step - loss: 0.3438 - accuracy: 0.8727
Epoch 6/10
111/111 [==============================] - 132s 1s/step - loss: 0.3807 - accuracy: 0.8798
Epoch 7/10
111/111 [==============================] - 132s 1s/step - loss: 0.2515 - accuracy: 0.9188
Epoch 8/10
111/111 [==============================] - 132s 1s/step - loss: 0.2258 - accuracy: 0.9254
Epoch 9/10
111/111 [==============================] - 133s 1s/step - loss: 0.1686 - accuracy: 0.9400
Epoch 10/10
111/111 [==============================] - 132s 1s/step - loss: 0.2267 - accuracy: 0.924

In [ ]:
!pip install tflite-model-maker

     |████████████████████████████████| 194kB 18.1MB/s 
     |████████████████████████████████| 1.0MB 39.2MB/s 
     |████████████████████████████████| 1.2MB 52.5MB/s 
     |████████████████████████████████| 5.5MB 51.1MB/s 
     |████████████████████████████████| 71kB 10.6MB/s 
     |████████████████████████████████| 645kB 46.7MB/s 
     |████████████████████████████████| 849kB 51.8MB/s 
     |████████████████████████████████| 92kB 10.2MB/s 
     |████████████████████████████████| 194kB 50.7MB/s 
     |████████████████████████████████| 102kB 15.7MB/s 
     |████████████████████████████████| 174kB 51.2MB/s 
     |████████████████████████████████| 37.6MB 79kB/s 
     |████████████████████████████████| 358kB 51.0MB/s 
     |████████████████████████████████| 706kB 48.2MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115928 sha256=360311412f2a5a6048eade1a4966fdbf212c6103552d0e9041cde9b9ba30366e
  Stored in directory: /root/.cache/pip/wheels/af/19/30/1ea0cad502dc

In [ ]:
import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')
from tensorflow_examples.lite.model_maker.core.task.model_spec import mobilenet_v2_spec
from tensorflow_examples.lite.model_maker.core.task.model_spec import ImageModelSpec
from tensorflow_examples.lite.model_maker.core.data_util.image_dataloader import ImageClassifierDataLoader
from tensorflow_examples.lite.model_maker.core.task import image_classifier

import matplotlib.pyplot as plt

In [ ]:
traindata = ImageClassifierDataLoader.from_folder('/content/drive/MyDrive/train')
testdata=ImageClassifierDataLoader.from_folder('/content/drive/MyDrive/test')

INFO:tensorflow:Load image with size: 3581, num_label: 36, labels: apple, banana, beetroot, bell pepper, cabbage, capsicum, carrot, cauliflower, chilli pepper, corn, cucumber, eggplant, garlic, ginger, grapes, jalepeno, kiwi, lemon, lettuce, mango, onion, orange, paprika, pear, peas, pineapple, pomegranate, potato, raddish, soy beans, spinach, sweetcorn, sweetpotato, tomato, turnip, watermelon.
INFO:tensorflow:Load image with size: 359, num_label: 36, labels: apple, banana, beetroot, bell pepper, cabbage, capsicum, carrot, cauliflower, chilli pepper, corn, cucumber, eggplant, garlic, ginger, grapes, jalepeno, kiwi, lemon, lettuce, mango, onion, orange, paprika, pear, peas, pineapple, pomegranate, potato, raddish, soy beans, spinach, sweetcorn, sweetpotato, tomato, turnip, watermelon.


In [ ]:
model = image_classifier.create(traindata)

INFO:tensorflow:Retraining the models...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hub_keras_layer_v1v2 (HubKer (None, 1280)              3413024   
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 36)                46116     
Total params: 3,459,140
Trainable params: 46,116
Non-trainable params: 3,413,024
_________________________________________________________________
None
Epoch 1/5
111/111 [==============================] - 219s 2s/step - loss: 3.0936 - accuracy: 0.2503
Epoch 2/5
111/111 [==============================] - 217s 2s/step - loss: 1.6453 - accuracy: 0.6949
Epoch 3/5
111/111 [==============================] - 218s 2s/step - loss: 1.4466 - accuracy: 0.7574
Epoch 4/5
111/111 [======

In [ ]:
loss, accuracy = model.evaluate(testdata)

12/12 [==============================] - 29s 2s/step - loss: 0.9547 - accuracy: 0.9471


In [ ]:
model.export(export_dir='/content/drive/MyDrive/model', with_metadata=False)